Case Study по модулю SQL
Исполнитель - Мирахмедова Парвина


**Блок 1. Создание схемы и таблиц**

In [ ]:
import json
import pandas as pd
from sqlalchemy import text

from connect import set_connection

SCHEMA = "adw_works"

def create_tables():
    try:
        # читаем содержимое файла 'queries/tables.sql'
        with open('queries/tables.sql') as f:
            tables_query = f.read()
        
        # создаём схему и таблицы
        with set_connection() as psg:
            psg.execute(text(tables_query))
            psg.commit()
        
        print("Tables created successfully")
    except Exception as e:
        print(e)

create_tables()



def read_xl(sheet_name, columns_dict):
    temp_df = pd.read_excel(
        'source/adventure_works.xlsx',
        sheet_name=sheet_name,
        usecols=columns_dict.keys()
    ).rename(columns=columns_dict)
    return temp_df

with open('columns.json') as f:
    tables_dict = json.load(f) 

def insert_to_db(temp_df, tbl_name):
    with set_connection() as psg:
        temp_df.to_sql(
            schema=SCHEMA,
            name=tbl_name,
            con=psg,
            index=False,
            if_exists='append'
        )

def etl(sheet_name, columns_dict, tbl_name):
    print(f"inserting data to {tbl_name}...")
    temp_df = read_xl(sheet_name, columns_dict)
    insert_to_db(temp_df, tbl_name)
    
for k, v in tables_dict.items():
    etl(k, v["columns"], v["table_name"])



**Блок 2. Аналитические задачи**

**Секция 1. Анализ клиентов**
***Сегментация по доходу***: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query ="""
SELECT 
    occupation,
    COUNT(customer_key) AS number_of_customers,
    round(AVG(yearly_income),2) AS avg_income
FROM 
    adw_works.customers
GROUP BY 
    occupation
ORDER BY 
    occupation;
"""


with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,occupation,number_of_customers,avg_income
0,Clerical,2928,30710.38
1,Management,3075,92325.20
2,Manual,2384,16451.34
3,Professional,5520,74184.78
4,Skilled Manual,4577,51715.10


Вывод.Самый высокий ежегодный доход у представителей менеджмента, а самый низкий - у представителей ручного труда.

**Секция 1. Анализ клиентов**
***Семейный профиль***: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: *has_children* (где 1 означает - имеет детей и 0 - не имеет детей), *pct_of_customer_base*.

In [8]:

query ="""SELECT 
    number_of_children,
    round(COUNT(customer_key) * 100.0 / (SELECT COUNT(*) FROM adw_works.customers),2) AS pct_of_customer_base
FROM 
    adw_works.customers
GROUP BY 
    number_of_children
ORDER BY 
    number_of_children;
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,number_of_children,pct_of_customer_base
0,0,60.14
1,1,13.31
2,2,8.92
3,3,6.51
4,4,5.89
5,5,5.23


Вывод. Покупатели без детей составляют 60% всех покупателей, 40% - покупатели с детьми, где большая часть 13% - это покупатели с одним ребенком.

**Секция 1. Анализ клиентов **
***Высокодоходные клиенты***: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле `SalesAmount`). Итоговая таблица должна содержать следующие поля: *customer_key*, *customer_name*, *total_purchase*.

In [11]:
query = """SELECT 
    c.customer_key, 
    c.full_name AS customer_name, 
    round(SUM(s.sales_amount),2) AS total_purchase
FROM 
    adw_works.customers c
JOIN 
    adw_works.sales s ON c.customer_key = s.customer_key
GROUP BY 
    c.customer_key, c.full_name
ORDER BY 
    total_purchase DESC
LIMIT 10;
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,customer_key,customer_name,total_purchase
0,12301,Nichole Nara,13295.38
1,12132,Kaitlyn Henderson,13294.27
2,12308,Margaret He,13269.27
3,12131,Randall Dominguez,13265.99
4,12300,Adriana Gonzalez,13242.70
5,12321,Rosa Hu,13215.65
6,12124,Brandi Gill,13195.64
7,12307,Brad She,13173.19
8,12296,Francisco Sara,13164.64
9,11433,Maurice Shan,12909.67


Вывод. Наибольшая сумма покупок среди покупателей составляет больше 13 тысяч $, т.е. покупатели готовы тратить не более 13 тысяч.

 **Секция 1. Анализ клиентов**
 ***Влияние семейного положения***: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (*MaritalStatus*) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: *year*, *marital_status*, *avg_sales_amount*.

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query = """SELECT 
    cast(EXTRACT(YEAR FROM s.order_date) as INT) AS year,
    c.marital_status,                       
    round(AVG(s.sales_amount),2) AS avg_sales_amount 
FROM 
    adw_works.customers c
JOIN 
    adw_works.sales s ON c.customer_key = s.customer_key
GROUP BY 
    year, c.marital_status
ORDER BY 
    year, marital_status;
    """
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df


intizorparvina


,year,marital_status,avg_sales_amount
0,2001,M,3245.03
1,2001,S,3203.84
2,2002,M,2397.07
3,2002,S,2482.13
4,2003,M,378.56
5,2003,S,427.78
6,2004,M,290.64
7,2004,S,318.05


Вывод. В 2001 и 2002 годах независимо от семейного положения покупатели потратили приблизительно одинаковое количество денег, чуть больше женатые в 2001 и и чуть больше свободные - в 2002 годах. В 2003 и 2004 затраты значительно уменьшились по сравнению с предыдущими годами и при этом, больше стали тратить назамужние/неженатые.

**Секция 2. Анализ продаж** 
 ***Ежемесячные продажи***: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *sales_count* (количество продаж), *sales_amount*.  

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query = """
SELECT
    CAST(EXTRACT(YEAR FROM c.first_purchase_date) AS INT) AS year,                  
    TO_CHAR(c.first_purchase_date, 'YYYYMM') AS monthkey,               
    TO_CHAR(c.first_purchase_date, 'FMMonth') AS month_name,            
    COUNT(*) AS sales_count,                                             
    round(SUM(s.sales_amount),1) AS sales_amount                                  
FROM
    adw_works.customers c                                                
JOIN
    adw_works.sales s ON c.customer_key = s.customer_key                
WHERE
    EXTRACT(YEAR FROM c.first_purchase_date) IN (2003, 2004)            
GROUP BY
    EXTRACT(YEAR FROM c.first_purchase_date),                            
    TO_CHAR(c.first_purchase_date, 'YYYYMM'),                           
    TO_CHAR(c.first_purchase_date, 'FMMonth')                           
ORDER BY
    year, monthkey;                                                     
                                             
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,year,monthkey,month_name,sales_count,sales_amount
0,2003,200301,January,1067,949602.8
1,2003,200302,February,1152,1006238.4
2,2003,200303,March,1191,1009850.7
3,2003,200304,April,1109,971965.8
4,2003,200305,May,1324,1161483.1
5,2003,200306,June,1253,1183195.9
6,2003,200307,July,585,343439.5
7,2003,200308,August,5346,397916.9
8,2003,200309,September,3613,377478.2
9,2003,200310,October,3463,436072.9


Вывод. Наибольшее количество продаж в 2003 году отмечалось в августе, а наименьшее - в июле. В 2004 - наибольшее количество продаж отмечалось в мае, а наименьшее - также в июле.

**Секция 2. Анализ продаж** 
 ***Продажи по регионам***: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: *region*, *sales_count*, *sales_amount*.

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 
query = """ 
SELECT 
    t.country AS region,
    COUNT(s.sales_amount) AS sales_count,
    round(SUM(s.sales_amount),1) AS sales_amount
FROM 
    adw_works.sales s
JOIN 
    adw_works.territory t ON s.territory_key = t.territory_key
GROUP BY 
    t.country
ORDER BY 
    t.country;

"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,region,sales_count,sales_amount
0,Australia,13345,9061000.6
1,Canada,7620,1977844.9
2,France,5558,2644017.7
3,Germany,5625,2894312.3
4,United Kingdom,6906,3391712.2
5,United States,21344,9389789.5


Вывод. Больше всего покупок из США и их вклад в сумму продаж самый большой, наименьший вклад вносят покупки из Франции,их же и наименьшее количество.